In [23]:
from supabase import create_client
from utils.env import SUPABASE_URL, SUPABASE_KEY


supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

def _get_popularity_distribution(artist):
    filter_response = supabase.table("songs")\
        .select("metrics(popularity)")\
        .like('artist_names', f'%{artist}%')\
        .execute()
    results = [x.get('metrics').get('popularity') for x in filter_response.data]
    return (min(results), round(sum(results)/len(results)), max(results))

resp = _get_popularity_distribution("Drake")
print(resp)

2024-01-09 22:10:23,450:INFO - HTTP Request: GET https://snuqfngjrznygycntzst.supabase.co/rest/v1/songs?select=metrics%28popularity%29&artist_names=like.%25Drake%25 "HTTP/1.1 200 OK"


(33, 65, 92)
